# Lab | Web Scraping

Welcome to the "Books to Scrape" Web Scraping Adventure Lab!

**Objective**

In this lab, we will embark on a mission to unearth valuable insights from the data available on Books to Scrape, an online platform showcasing a wide variety of books. As data analyst, you have been tasked with scraping a specific subset of book data from Books to Scrape to assist publishing companies in understanding the landscape of highly-rated books across different genres. Your insights will help shape future book marketing strategies and publishing decisions.

**Background**

In a world where data has become the new currency, businesses are leveraging big data to make informed decisions that drive success and profitability. The publishing industry, much like others, utilizes data analytics to understand market trends, reader preferences, and the performance of books based on factors such as genre, author, and ratings. Books to Scrape serves as a rich source of such data, offering detailed information about a diverse range of books, making it an ideal platform for extracting insights to aid in informed decision-making within the literary world.

**Task**

Your task is to create a Python script using BeautifulSoup and pandas to scrape Books to Scrape book data, focusing on book ratings and genres. The script should be able to filter books with ratings above a certain threshold and in specific genres. Additionally, the script should structure the scraped data in a tabular format using pandas for further analysis.

**Expected Outcome**

A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`. The function should scrape book data from the "Books to Scrape" website and return a `pandas` DataFrame with the following columns:

**Expected Outcome**

- A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`.
- The function should return a DataFrame with the following columns:
  - **UPC**: The Universal Product Code (UPC) of the book.
  - **Title**: The title of the book.
  - **Price (£)**: The price of the book in pounds.
  - **Rating**: The rating of the book (1-5 stars).
  - **Genre**: The genre of the book.
  - **Availability**: Whether the book is in stock or not.
  - **Description**: A brief description or product description of the book (if available).
  
You will execute this script to scrape data for books with a minimum rating of `4.0 and above` and a maximum price of `£20`. 

Remember to experiment with different ratings and prices to ensure your code is versatile and can handle various searches effectively!

**Resources**

- [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/index.html)
- [Books to Scrape](https://books.toscrape.com/)


**Hint**

Your first mission is to familiarize yourself with the **Books to Scrape** website. Navigate to [Books to Scrape](http://books.toscrape.com/) and explore the available books to understand their layout and structure. 

Next, think about how you can set parameters for your data extraction:

- **Minimum Rating**: Focus on books with a rating of 4.0 and above.
- **Maximum Price**: Filter for books priced up to £20.

After reviewing the site, you can construct a plan for scraping relevant data. Pay attention to the details displayed for each book, including the title, price, rating, and availability. This will help you identify the correct HTML elements to target with your scraping script.

Make sure to build your scraping URL and logic based on the patterns you observe in the HTML structure of the book listings!


---

**Best of luck! Immerse yourself in the world of books, and may the data be with you!**

**Important Note**:

In the fast-changing online world, websites often update and change their structures. When you try this lab, the **Books to Scrape** website might differ from what you expect.

If you encounter issues due to these changes, like new rules or obstacles preventing data extraction, don’t worry! Get creative.

You can choose another website that interests you and is suitable for scraping data. Options like Wikipedia, The New York Times, or even library databases are great alternatives. The main goal remains the same: extract useful data and enhance your web scraping skills while exploring a source of information you enjoy. This is your opportunity to practice and adapt to different web environments!

In [15]:
# Your solution goes here
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

# Base URL for the site
BASE_URL = "http://books.toscrape.com/"

# Convert rating text to number
RATING_MAP = {
    "One": 1,
    "Two": 2,
    "Three": 3,
    "Four": 4,
    "Five": 5
}

# Builds a full link from a relative one
def get_full_url(relative_url):
    full_url = BASE_URL + relative_url.replace('../../', '')
    print("Detail URL:", full_url)
    return full_url

# Extracts data for books from one page
def extract_book_data(book_soup, min_rating, max_price):
    books = []

    for article in book_soup.select('article.product_pod'):
        print("\n--- New Book ---")

        # Extract and convert rating
        rating_classes = article.select_one('p.star-rating')['class']
        rating_str = [cls for cls in rating_classes if cls != 'star-rating'][0]
        rating = RATING_MAP.get(rating_str, 0)
        print("Rating:", rating)

        # Get price text and convert to float
        price_text = article.select_one('p.price_color').text.strip()
        price_match = re.findall(r'\d+\.\d+', price_text)
        if not price_match:
            print("Price format error, skipping")
            continue
        price = float(price_match[0])
        print("Price:", price)

        # Skip books that don't match filters
        if rating < min_rating:
            print("Skipped: rating too low")
            continue
        if price > max_price:
            print("Skipped: price too high")
            continue

        # Load book detail page
        detail_url = get_full_url(article.h3.a['href'])
        detail_response = requests.get(detail_url)
        detail_response.encoding = 'utf-8'
        detail_soup = BeautifulSoup(detail_response.text, 'html.parser')

        # Extract title
        title = detail_soup.h1.text.strip()
        print("Title:", title)

        # Extract data from product table
        table = detail_soup.select_one('table.table.table-striped')
        if not table:
            print("Table not found, skipping book.")
            continue

        upc_tag = table.find('th', string="UPC")
        availability_tag = table.find('th', string="Availability")

        if not upc_tag or not availability_tag:
            print("Required data missing, skipping book.")
            continue

        upc = upc_tag.find_next_sibling('td').text
        availability = availability_tag.find_next_sibling('td').text.strip()
        print("UPC:", upc)
        print("Availability:", availability)

        # Extract genre from breadcrumbs
        genre = detail_soup.select('ul.breadcrumb li')[2].text.strip()
        print("Genre:", genre)

        # Extract description (if available)
        description_tag = detail_soup.select_one('div#product_description')
        if description_tag:
            description = description_tag.find_next_sibling('p').text.strip()
        else:
            description = ''
        print("Description:", description[:60], "...")

        # Save book data
        books.append({
            "UPC": upc,
            "Title": title,
            "Price (£)": price,
            "Rating": rating,
            "Genre": genre,
            "Availability": availability,
            "Description": description
        })

    return books

# Go through all pages and collect data
def scrape_books(min_rating=4, max_price=20):
    page = 1
    all_books = []

    while True:
        # Build full page URL
        url = f"{BASE_URL}catalogue/page-{page}.html"
        print(f"\n=== Page {page} ===\nFetching:", url)
        res = requests.get(url)
        res.encoding = 'utf-8'

        # Stop if no more pages
        if res.status_code != 200:
            print("No more pages or request failed.")
            break

        soup = BeautifulSoup(res.text, 'html.parser')
        books_on_page = extract_book_data(soup, min_rating, max_price)

        # Stop if no books matched filters
        if not books_on_page:
            print("No matching books on this page.")
            break

        all_books.extend(books_on_page)
        page += 1

        # Optional: limit scraping to first 3 pages
        if page > 3:
            print("Page limit reached.")
            break

    # Convert to DataFrame
    df = pd.DataFrame(all_books)
    print("\nFinal DataFrame Preview:")
    print(df.head())
    return df

# Run the scraper
df = scrape_books(min_rating=4, max_price=20)

# Optional: Format DataFrame display for terminal (VS Code or CLI)
pd.set_option('display.max_columns', None)     # Show all columns
pd.set_option('display.width', 200)            # Avoid line breaks
pd.set_option('display.colheader_justify', 'left')  # Align headers left

# Show first 10 rows
print("\nPreview of books found:")
print(df.head(10))



=== Page 1 ===
Fetching: http://books.toscrape.com/catalogue/page-1.html

--- New Book ---
Rating: 3
Price: 51.77
Skipped: rating too low

--- New Book ---
Rating: 1
Price: 53.74
Skipped: rating too low

--- New Book ---
Rating: 1
Price: 50.1
Skipped: rating too low

--- New Book ---
Rating: 4
Price: 47.82
Skipped: price too high

--- New Book ---
Rating: 5
Price: 54.23
Skipped: price too high

--- New Book ---
Rating: 1
Price: 22.65
Skipped: rating too low

--- New Book ---
Rating: 4
Price: 33.34
Skipped: price too high

--- New Book ---
Rating: 3
Price: 17.93
Skipped: rating too low

--- New Book ---
Rating: 4
Price: 22.6
Skipped: price too high

--- New Book ---
Rating: 1
Price: 52.15
Skipped: rating too low

--- New Book ---
Rating: 2
Price: 13.99
Skipped: rating too low

--- New Book ---
Rating: 4
Price: 20.66
Skipped: price too high

--- New Book ---
Rating: 5
Price: 17.46
Detail URL: http://books.toscrape.com/set-me-free_988/index.html
Title: 404 Not Found
Table not found, skip